In [11]:
from flask import Flask, request,render_template, jsonify
from flask_cors import CORS
from pydub import AudioSegment
from sqlalchemy import create_engine,text
from dotenv import load_dotenv
from openai import OpenAI

import base64,random, io, re, os, sys
import pandas as pd 
import numpy as np 


In [ ]:
load_dotenv()

In [13]:
class SingletonMeta(type):
    """
    A Singleton metaclass that ensures a class has only one instance.
    """
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

class AppState(metaclass=SingletonMeta):
    '''
    This is AiMl App State Please create a instence of the class to use it before running the server.
    '''
    def __init__(self):
        # Database initialization
        self.db_path = "data/test_sqldb.db"
        self.db_url = f"sqlite:///{self.db_path}"
        self.db = create_engine(self.db_url)
        
        # Meta Data of db
        self.metaData = pd.read_csv('data/metaData.csv')
        # OpenAI client initialization
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        

    def run_query(self,query):
        with self.db.connect() as connection:
            result = connection.execute(text(query))
            return result.fetchall()
    
    def get_sql_prompt(self,user_query,dBType="SQLite",topk=10): 
        
        return f'''
                <Your Persona>
                        You are an SQL expert that can create any complex {dBType} queries, for SQL Database, given a textual user query.
                </Your Persona>

                # User Question: A natural language question about the real estate data in a specific database.
                <User Question:>{user_query}</User Question>
                
                ## Critical Rules to Follow:
                <Rules>
                    - Only Give {dBType} Sql query in your output/response that is ready to run on the database.
                    - DO NOT refer to columns that do not exist in the tag <DB dictionary>
                    - Always use Columns which are mentioned in the tag <DB dictionary> Below!
                    - If the query is about listing houses then Always write PropertyID in the SQL query.
                    - Given an input question, create a syntactically correct {dBType} query to run, then look at the results of the query and return the answer.   
                    - Unless the user specifies a specific number of examples they wish to obtain, Always limit your query to at most {topk} results.
                    - Never query for all the columns from a specific table, only ask for the relevant columns given the question.
                    - DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
                    - Handle different types of user questions (e.g., filtering, aggregation, joins).
                    - Use efficient query structures (e.g., SELECT, FROM, WHERE, JOIN, GROUP BY, ORDER BY, LIKE, SUM, etc.) to optimize performance.
                </Rules>
                # Example so you can understand.
                <sample DB DATA>
                    PropertyID,StatusChangeDate,SitusCounty,Township,MLSListingAddress,MLSListingCity,MLSListingState,MLSListingZip,MLSListingCountyFIPS,AssessorLastSaleAmount,MarketValue,MarketValueDate,AvgMarketPricePerSqFt,ListingDate,LatestListingPrice,SpecialListingConditions,OriginalListingDate,OriginalListingPrice,NumberOfLeaseOption,LeaseTerm,LeaseDetails,MLSPropertyType,MLSPropertySubType,OwnershipDescription,Latitude,Longitude,APNFormatted,LegalDetails,LegalSubdivision,DaysOnMarket,ListingAgentFullName,ListingAgentPreferredPhone,ListingAgentEmail,ListingOfficeName,ListingOfficePhone,ListingOfficeEmail,PublicListingRemarks,HomeWarrantyYN,YearBuilt,NewConstructionYN,AdditionalParcelsYN,NumberOfLots,LotSizeSquareFeet,LotSizeAcres,LotSizeSource,LotDimensions,LotFeatureList,FrontageLength,FrontageType,FrontageRoadType,LivingAreaSquareFeet,LivingAreaSource,Levels,Stories,BuildingStoriesTotal,BuildingKeywords,BuildingAreaTotal,NumberOfUnitsTotal,PropertyAttachedYN,OtherStructures,BedroomsTotal,BathroomsFull,BathroomsHalf,BathroomsQuarter,BasementDetails,BelowGradeSquareFeet,BasementTotalSqFt,BasementFinishedSqFt,BasementUnfinishedSqFt,PropertyConditionDetails,RepairsYN,RepairsDetails,Disclosures,ConstructionMaterials,GarageYN,AttachedGarageYN,GarageSpaces,CarportYN,CarportSpaces,ParkingDetails,ParkingOther,OpenParkingSpaces,ParkingTotal,PoolPrivateYN,PoolDetails,Occupancy,ViewYN,ViewDetails,Topography,HeatingYN,HeatingDetails,CoolingYN,CoolingDetails,FireplaceYN,FireplaceDetails,FireplaceNumber,FoundationDetails,RoofDetails,ArchitecturalStyleDetails,PatioAndPorchDetails,UtilitiesDetails,ElectricDetails,WaterSourceDetails,SewerDetails,GasDescriptionDetails,OtherEquipmentIncluded,LaundryDetails,AppliancesDetails,InteriorDetails,ExteriorDetails,FencingDetails,PetsAllowedYN,HorseZoningYN,SeniorCommunityYN,WaterfrontYN,WaterfrontDetails,SchoolElementary,SchoolElementaryDistrict,SchoolMiddle,SchoolMiddleDistrict,SchoolHigh,SchoolHighDistrict,LandLeaseYN,LandLeaseAmount,LandLeaseAmountFrequency,RentControlYN,
                    361238494,2024-07-22,Orange,,27211 Capote De Paseo C,San Juan Capistrano,CA,92675,6059,470000.0,747750.0,6/25/2024,689.0,7/22/2024,748000.0,Standard,7/22/2024,748000.0,0.0,,,Residential,Single Family Residence,,33.4927,-117.656,666-034-20,TR 5514 BLK 6 LOT 20 & 1/24TH UND INT IN LOT 25 BLK 6 TR 5514 &1/180TH UND INT IN LOTS A THRU I TR 5514,San Juan Hills (JH),0.0,Melanie McLaird,949-662-5533,,Pacific Sotheby's Int'l Realty,,,"Nestled amidst lush greenery in the highly sought-after 55+ community of San Juan Hills West, this exquisite single-level home welcomes you with private patios and a generous floor plan. The residence is light and bright throughout, thanks to numerous skylights. It offers plenty of storage space, air conditioning, and a walkout attic.  The open living area seamlessly connects to the kitchen, which features a picture window that fills the space with natural light. The dining room, adorned with tile flooring, also boasts abundant light. The primary suite includes a large walk-in closet. A sliding door leads to your second private patio, where you can sip your morning coffee and enjoy a quiet moment. An indoor laundry area is conveniently tucked away in a closet.  Residents can enjoy the community pool and clubhouse. The home is conveniently located near the charming and vibrant historical Mission San Juan Capistrano, downtown dining and shopping, and Dana Point Harbor. Don't miss the opportunity to make this incredible property yours!",N,1972.0,N,N,,1324.0,0.0304,Assessor,,Greenbelt,,,,1300.0,Assessor,One,,1.0,,,1.0,Y,,2.0,2.0,,,,,,0.0,,,N,,,,,N,2.0,,,,,,2.0,N,"Community,In Ground",,Y,"Neighborhood,Pool",,Y,Central,Y,Central Air,Y,Living Room,,,,,Patio,"Cable Available,Electricity Available,Phone Available,Water Available",,Public,,,,"Dryer Included,In Closet,Inside,Washer Included","Dishwasher,Electric Oven,Freezer,Gas Cooktop,Microwave,Refrigerator","Ceiling Fan(s),Granite Counters,Open Floorplan,Pull Down Stairs to Attic,Recessed Lighting,Storage",Rain Gutters,Block,,,Y,N,,Harold J. Ambuehl Elementary School,Capistrano Unified School District,Marco Forster Middle School,Capistrano Unified School District,San Juan Hills High School,Capistrano Unified School District,N,,,N
                    354096350,2024-07-22,,,1109 Rivington,Irvine,CA,92612,6059,,,,,5/15/2024,4647.0,,5/15/2024,4997.0,,,Association Dues,Residential Lease,Condominium,,,,,,Other (OTHR),33.0,Rajshekhar Kolmi,,,K & D Associates,,,"Welcome to 1109 Rivington, A brand-new never lived-in, luxurious, and spacious two-bedroom condo situated in the coveted Central Park West community at the Lexington building. This lovely interior condo is located on the 1st floor, Its features include a spacious open floor plan where the living and dining rooms seamlessly connect with an extended outdoor deck for entertainment and pleasure. Amenities include a full-sized stacked washer and dryer in an enclosed closet,  Recessed LED lighting throughout, luxury vinyl flooring and plush carpet in the bedroom. Cheff inspired kitchen offers high-end GE Monogram appliances, quartz countertops with subway backsplash and an Island. A spacious guest bathroom provides privacy of your en-suite. Building amenities include ample outdoor space with a ping-pong table, foosball table, numerous cabanas, multiple flat-screen TV areas, BBQ stations, outdoor workplaces, a dog park and a mailroom, just to name a few. The Central Park West community also offers a gym, resort-style pools, recreation rooms, a pickleball court, and more. Trash service and water are included in the rent, while the tenant is responsible for electricity and gas.with EV chargers available at the user's cost in the guest parking area. Experience luxury living at one of the newest communities located in the heart of Irvine, with close proximity to John Wayne Airport, world-class shopping, and dining.",N,2024.0,Y,N,,100.0,0.0023,Builder,,16-20 Units/Acre,,,,1800.0,,One,,6.0,,,10.0,Y,,2.0,2.0,1.0,,,,,,,,N,,,,,N,2.0,,2.0,Detached Carport,,,4.0,N,Association,,N,,,N,,Y,"Central Air,ENERGY STAR Qualified Equipment",N,,,,,,,,,Public,Public Sewer,,,"Dryer Included,Washer Included","6 Burner Stove,Dishwasher,Gas Range,Gas Water Heater,Microwave",,,,Call,,N,N,,,,,,,Irvine Unified,N,,,N,
                    236308051,2024-07-22,Orange,,34032 Selva 96,Dana Point,CA,92629,6059,549000.0,878750.0,8/22/2023,806.0,8/3/2023,4000.0,,8/3/2023,3700.0,,,"Association Dues,Gardener,Pool",Residential Lease,Condominium,,33.4674,-117.712,939-411-21,"TR 11799 LOT 7 UNIT 96 OF PROJECT LOCATED ON AP 672-094-11 TOGETHER WITH AN UND 1/296 INT IN LOTS 1, 6, 7 & POR 8 PER L.L.",Niguel Beach Terrace (NBT),353.0,Joy Cuda,,,Beach Properties,,,"Furnished. August 2024 is $4000. September 2024  to July 1st 2025 at $3500.mo ,July 1st, 2025 at $4000, YEAR LEASE $3900. plus Utilities and Cable with Internet and WIFI.  All new bed linens ,pillows, and spreads ..Sleeps 4 one queen and two twins. . Freshly painted in every room with Travertine floors except the two bedroom which have vinyl plank flooring. Two bathrooms have travertine plus the two patios have travertine also. New Stacked washer dryer in second bathroom . Ceiling fans in each room, no A/C and nice vertical blinds. Easy access to the ASSIGNED parking space # 507 just across from the front patio, the second patio is private just off of the dining area. Kitchen includes Stainless refrigerator, dishwasher, black Range and microwave with nicely finished kitchen cabinets.  No weekly rentals, 30 days or more.",N,1982.0,N,N,,0.0,0.0,Not Taped,,,,,,873.0,Public Records,One,,1.0,,,296.0,Y,,2.0,2.0,,,,,,0.0,,,N,,,,,N,0.0,,0.0,"Assigned,Parking Space",,,1.0,N,"Association,Community",,Y,Trees/Woods,,Y,Central,N,,N,,,,,,,,,Public,Public Sewer,,,"Dryer Included,Inside,Stackable,Washer Included",,,,,"Call,Dogs OK,No",,N,N,,R. H. Dana Language & Science Academy,Capistrano Unified School District,Marco Forster Middle School,Capistrano Unified School District,Dana Hills High School,Capistrano Unified School District,N,,,N,
                </sample DB DATA>
                # Example so you can understand.
                <Example>
                    "input1": "Are there houses available for 50k?",
                    "query produced": "SELECT PropertyID, LatestListingPrice, BedroomsTotal FROM properties WHERE LatestListingPrice <= 50000 LIMIT {topk};"            
                
                    "input2": "I need a house with 3 bedrooms. Please provide me details",
                    "query produced": "SELECT PropertyID, LatestListingPrice, MLSListingAddress, BedroomsTotal, BathroomsFull, AvgMarketPricePerSqFt FROM properties WHERE BedroomsTotal = 3 LIMIT {topk};"
                
                    "input3": "How many apartments are available near La Ventana?",
                    "query produced": "SELECT COUNT(*) FROM properties WHERE MLSListingCity = 'Laguna Beach' AND MLSPropertyType = 'Residential';"
            
                    "input4": "I need a house near the water body or lake or beach"
                    "query produced": "SELECT PropertyID, MLSListingAddress, LatestListingPrice, FROM properties WHERE (WaterfrontDetails LIKE '%ocean%' OR WaterfrontDetails LIKE '%beach%' OR WaterfrontDetails LIKE '%sea%') LIMIT {topk};"

                    "input5": "Need some houses with good view"
                    "query produced": "SELECT PropertyID, MLSListingAddress, LatestListingPrice, BedroomsTotal, ViewDetails FROM properties WHERE ViewYN = 'Y' AND (ViewDetails LIKE '%ocean%' OR ViewDetails LIKE '%city%' OR ViewDetails LIKE '%excellent%' OR ViewDetails LIKE '%mountain%') LIMIT {topk};"
                
                    "input6": "Show me the houses with are newly constructed"
                    "query produced": "SELECT PropertyID, MLSListingAddress, YearBuilt FROM properties WHERE NewConstructionYN = 'Y' ORDER BY YearBuilt DESC LIMIT {topk};"

                    "input7": "Show a list of repaired as they are cheap."
                    "query produced": "SELECT PropertyID, MLSListingAddress, YearBuilt FROM properties WHERE RepairsYN = 'Y' ORDER BY LatestListingPrice DESC LIMIT {topk};"
                <Examples>

                #Below is DB dictionary Use it to Generate the Query and understand what the user is asking. Data Dictionary had discription of each column in the data base
                # Pay vary good attention to COLUMN NAMES and only give the Columns which are mentioned in the tag <DB dictionary> Below. DO NOT REFER to any column name that is not mentioned in the tag <DB dictionary>!
                <DB dictionary>
                        {self.metaData}
                </DB dictionary> '''

    def information_prompt(self,user_query,dBresults,sql):
        return f"""
                <Your Persona>
                    You are a Real Estate Agent. Who has access to A real estate database. You provide Assistance to the user's questions. 
                    Your role is to provide solutions to users' questions by crafting and executing SQL queries based on their inquiries.
                    You recive a user question, a query and response based on this information you provide Educated analysis and assistent.
                    
                    I need you to respond to queries as if you are actively searching for the information. 
                    Your responses should sound natural and conversational, implying that you are finding the results yourself. 
                        Here are some examples:
                        Instead of saying 'The database says this,' you can say:
                                'I found these results that might interest you.'
                                'I was able to narrow down the information to this and that.'
                                'Oooh, I get it, this might be what you are looking for!'
                </Your Persona>

                ## Critical Rules to Follow:
                <Rules>
                    - Your role is to convey the results of the database accurately in a conversational manner. Do not fabricate any information.
                    - NEVER invent, fabricate, hallucinate, or modify data in any way.
                    - NEVER Mention MLS Listing ID in your answer.
                    - If the user asks for NaN or NULL values, return an empty string.
                    - You MUST reproduce the Database Output exactly as provided, without any changes.
                    - If the Database Output is empty, consists only of column headers, or contains any variation of "No results found", you MUST respond ONLY with: "Sorry, I could not find any properties matching your requirements."
                    - Do not attempt to interpret, summarize, or analyze the data unless explicitly asked to do so by the user after you've presented the raw data.
                    - If the database output/result is empty, respond accordingly. Inform the user based on the question they asked, and let them know you were unable to find the information.
                </Rules>

                ## Input you recived:
                <Input>
                    # The SQL query that ran on database and get the results below.
                    <SQL Query>{sql}</SQL Query>
                    
                    # Database results/Output from the Database:
                    <results>{dBresults}</results>

                    # Question asked which resulted in the SQL query you recived: 
                    <userQuestion>{user_query}</userQuestion>
                </Input>

                ## Response Format (Must be followed EXACTLY and tags must also be included in the output as are mentioned below. Do not put any information outside tags put full response in <response> tag. Put Summary of the information in <summary> tag):
                <Output Format>
                    <response>
                            Respond in a conversational way and give Answer to the query. Provide the findings/results you got from real estate database to the user.
                    </response>
                    <summary>Provide one liner Summrized your response in this tag</summary>
                </Output Format>
               """
    def information_exception_promp(self,user_query): 
        return f"""
                    You are a helpful real estate assistant specialising in data analysis. You answer generic qustions about the real state database. You maybe called when the sql query is incorrect or an exception is raised.
                    if the quesrtion is about fingind a house in the real estate database, You inform the user you were unabel to find the informaton and also try to tell the user to improve your query or put question in more details or other words.
                    Your task is to:
                    
                    You must add a concise summary of your Answer to the question below: Always give a concise summary of your answer. The summary must match your answer exactly.

                    ## Response Format (Must be followed EXACTLY and tags must also be included in the output as are mentioned below. Do not put any information outside tags put full response in <response> tag. Put Summary of the information in <summary> tag):
                    <Output Format>
                        <response>
                                Respond in a conversational way and give Answer to the query. Provide the findings/results you got from real estate database to the user.
                        </response>
                        <summary>Provide one liner Summrized your response in this tag</summary>
                    </Output Format>
                    
                    #User Question: A natural language question about a specific topic.
                    Question: {user_query} """

In [14]:
def transcribe(audio_data) -> str:
    
    audio_blob_bytes = audio_data.read()

    # Load the audio blob using pydub
    audio = AudioSegment.from_file(io.BytesIO(audio_blob_bytes))

    # Adjust the audio properties (sample rate, channels, sample width)
    audio = audio.set_frame_rate(22050)
    audio = audio.set_channels(1)
    audio = audio.set_sample_width(2)  

    wav_io = io.BytesIO()
    audio.export(wav_io, format="wav")
    wav_io.seek(0) 
    
    directory = 'temp'
    file_path = os.path.join(directory, f'{random.randint(1, 10000)}_audio.wav')

    # Ensure the directory exists
    if not os.path.exists(directory):
        os.makedirs(directory)

    with open(file_path, 'wb') as f:
        f.write(wav_io.read())
   
    try:
        with open(file_path, "rb") as audio_file:
            # Use the client to create an audio transcription
            transcription = AppState().client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                language="en"
            )
    finally:
        if os.path.exists(file_path):
            os.remove(file_path)

    return transcription.text

def split_answer_summary(text: str) -> (str, str): # type: ignore
    # Use regex to find the <answer> and <summary> tags and their contents
    answer_match = re.search(r'<response>(.*?)</response>', text, re.DOTALL)
    summary_match = re.search(r'<summary>(.*?)</summary>', text, re.DOTALL)

    # Extract the contents between the tags
    answer = answer_match.group(1).strip() if answer_match else ""
    summary = summary_match.group(1).strip() if summary_match else ""

    return answer, summary

def extract_property_ids(results:list) -> list:
    try:
        elem = []
        for item in results:
            if isinstance(item[0], (int)):  # Check if the first element is a number
                elem.append(item[0])
        return elem
    except:
        return []
 
def fetch_property_details(property_ids: list):
    if not property_ids:
        return None

    try:
    # Dynamically generate the placeholders for the IN clause
        placeholders = ','.join([':p{}'.format(i) for i in range(len(property_ids))])
        query = f"""
            SELECT PropertyID, LatestListingPrice, MLSListingAddress, BedroomsTotal, BathroomsFull, AvgMarketPricePerSqFt, ListingAgentFullName, PhotosCount, PhotoKey, PhotoURLPrefix
            FROM properties 
            WHERE PropertyID IN ({placeholders})
        """
        parameters = {'p{}'.format(i): id_ for i, id_ in enumerate(property_ids)}

        with AppState().db.connect() as connection:
            result = connection.execute(text(query), parameters)
            df = pd.DataFrame(result.fetchall(), columns=[
                'PropertyID', 'LatestListingPrice', 'MLSListingAddress', 
                'BedroomsTotal', 'BathroomsFull', 'AvgMarketPricePerSqFt', 'ListingAgentFullName', 'PhotosCount', 'PhotoKey', 'PhotoURLPrefix'
            ])
        return df.to_json(orient='records')

    except Exception as e:
        return None


def openai_api(user_query, systemPrompt, max_tokens=1024, temperature=0.1, top_p=0.01, streaming=False):
    try:
        response = AppState().client.chat.completions.create(
        model=os.getenv("MODEL_NAME", "gpt-4o-mini"),
        messages=[
            {"role": "system", "content": systemPrompt},
            {"role": "user", "content": user_query}
        ], max_tokens=max_tokens, temperature=temperature, top_p=top_p, stream=streaming)

        return response.choices[0].message.content

    except Exception as e:
        return f"Exaeption in openAiApi {e}"
    
        
def textSqlApi(user_query):
    
    ids = []
    results=''
    response=''
    summary=''

    syatemPrompt = AppState().get_sql_prompt(user_query)
    sql = openai_api(user_query=user_query,systemPrompt=syatemPrompt).strip().replace("`", "").replace("sql\n", "")
    try:
        if sql:
            results = AppState().run_query(query=sql)
        # if results != '':
        #     results = ast.literal_eval(results)
        ids = extract_property_ids(results)
        print("\nResutls ->",results,"\nids->",ids)
        systemPrompt = AppState().information_prompt(user_query,results,sql)
        response = openai_api(user_query=user_query,systemPrompt=systemPrompt)
        
    except:
        print("Exception in DB->",sys.exc_info()) 
        systemPrompt = AppState().information_exception_promp(user_query)
        response = openai_api(user_query=user_query,systemPrompt=systemPrompt)
   

    print("\nsql->",sql,"LLm Output->",response)
    response,summary = split_answer_summary(response)
    
    return response,summary,ids

def ttsApi(text):
    
    spoken_response = AppState().client.audio.speech.create(
        model="tts-1",
        voice="shimmer",
        response_format="wav",
        input=text
    )

    buffer = io.BytesIO()
    for chunk in spoken_response.iter_bytes(chunk_size=4096):
        buffer.write(chunk)
    buffer.seek(0)

    return buffer 

def piplineAudio(audio_data):
    
    transcribed = transcribe(audio_data)
    print("whisper->",transcribed)
    message , text ,ids = textSqlApi(transcribed)
    audio_buffer = ttsApi(text)

    return audio_buffer, message, transcribed,text ,ids # Return audio buffer, message, and transcription

In [ ]:
appState = AppState()
app = Flask(__name__)
CORS(app)  

In [16]:
@app.route('/')
def index(): 
    return render_template('index.html')

In [17]:
@app.route('/upload_audio', methods=['POST'])
def upload_audio():
     

    audio_buffer, message, transcribe,summary, ids = piplineAudio(request.files['audio_data'])
      
    message = message.replace('\n', '<br>')

    fetxh_property = fetch_property_details(ids)
    print("fetxh_property->", fetxh_property)

    # Convert audio buffer to base64
    audio_base64 = base64.b64encode(audio_buffer.getvalue()).decode('utf-8')
     
    if(summary != ""):
      message = "<b>"+summary+"</b><br><br>"+message
       
    return jsonify({
        "message": message,
        "transcribed": transcribe,
        "audio": audio_base64,  # Return audio data as base64
        "property": fetxh_property
    })

In [18]:
@app.route('/upload_text', methods=['POST'])
def upload_text():
    
    data = request.get_json()
    print('input->',data['text'])
    
    message,summary ,ids = textSqlApi(data['text'])
    message = message.replace('\n', '<br>')
    
    print("details->", message)
    print("summary->",summary)
    print("ids->",ids)

    fetxh_property = fetch_property_details(ids)
    print("fetxh_property->",fetxh_property)
    
    if(summary != ""):
        message = "<b>"+summary+"</b><br><br>"+message
    
    return jsonify({"message": message,"property":fetxh_property})

In [ ]:
if __name__ == '__main__': 
    app.run(host='127.0.0.1', debug=False)